# Práctica 1 - Clasificación de palabras

In [6]:
import pandas as pd
from matplotlib.colors import ListedColormap
import sklearn.preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
#https://scikit-learn.org/stable/modules/model_evaluation.html
import matplotlib.pyplot as plt

import numpy as np
from numpy import mean

from sklearn.svm import SVC

## Preparación de los datos
En esta primera parte de la práctica vamos a preparar los datos para poder hacer la clasificación. Para ello haremos que cada muestra siga el siguiente formato: $C_1$ | $C_2$ | $C_3$ | .. | $C_n$ | $y$  (siendo $C$ una característica y $y$ el target).

In [7]:
#Auxiliar function that reformats df to one that is more fitting to Classification Problems
def reformat(dataFrame):
    if dataFrame.columns[0]=="catala":
        dataFrame['y']=0
    else:
        dataFrame['y']=1
    
    dataFrame.rename(columns={dataFrame.columns[0]: 'word'}, inplace=True)
    return dataFrame

#RawData, it has been a little bit formatted before reading the csv, to facilitate the process.
raw=pd.read_csv("data/data.csv")
#Split df
catala, angles= raw.filter(['catala'], axis=1), raw.filter(['angles'], axis=1)
#Reformating
catala=reformat(catala)
angles=reformat(angles)

#Merging
wordsDF=pd.concat([catala,angles], axis=0)
#Shuffle the rows
wordsDF = wordsDF.sample(frac=1).reset_index(drop=True)



### Caracterísiticas
Las características que hemos pensado que pueden ser útiles para la clasificación de las palabras son:
- Cantidad de caracteres (Númerica)
- Proporción de consonantes por vocal (consonantes / vocales) (Numérica)
- Contiene patrones o normas ortográficas de una lengua de las que vamos a clasificar?
    + Doble uso de vocal consecutivamente como es el caso del inglés (Categórica)
    + Acentos en caso de catalán (Categórica)
    + Contiene combinaciones de consonantes (consonant clusters) propias del inglés? (Categórica)

Referencias
- <Consonant_Clusters :https://www.aprendeinglessila.com/2013/09/consonantes-ingles-clusters/>
- <Frecuencia_De_Letras_Usadas_En_Catalan: https://es.sttmedia.com/frecuencias-de-letras-catalan>
- <Frecuencia_De_Letras_Usadas_En_Catalan: https://www3.nd.edu/~busiforc/handouts/cryptography/letterfrequencies.html>

Para añadir las columnas que representen estas características, hemos aplicado las siguientes funciones a las palabras.

In [8]:
#ratio de consonantes y vocales
def ratio (word):
    vocals=0
    for c in word:
        if isVocal(c):
            vocals+=1
    return round(vocals/len(word), 4)

#isVocal?
def isVocal(c):
    if(c=='a' or c=='e' or c=='i' or c=='o' or c=='u'):
        return True
    return 
#gotAccent?
def gotAccent(word):
    #List containing all possible accentuated chars from Catalan
    accentuatedChars=[ord('à'),ord('è'),ord('é'),ord('í'),ord('ò'),ord('ó'),ord('ú')]
    for c in word:
        if ord(c) in accentuatedChars:
            return 1
    return 0
def doubleVocal(word):
    ocurrences=["aa","ee","ii","oo","uu"]
    for oc in ocurrences:
        if word.find(oc)!=-1:
            return 1
    return 0

def enCC(word):
    ocurrences=["sch","spl","shr","squ","thr","spr","scr","sph","th","tw","sw","sk","sm"]
    for oc in ocurrences:
        if word.find(oc)!=-1:
            return 1
    return 0

#Features Adding
wordsDF['ratio']=wordsDF['word'].apply(ratio)
wordsDF['cantidadLetras']=wordsDF['word'].apply(len)
wordsDF['gotAccent']=wordsDF['word'].apply(gotAccent)
wordsDF['doubleVocal']=wordsDF['word'].apply(doubleVocal)
wordsDF['enCC']=wordsDF['word'].apply(enCC)
#Reorganize DF
wordsDF=wordsDF[['word','ratio','cantidadLetras','gotAccent','doubleVocal','enCC','y']]
wordsDF.to_csv('data/definitiveData.csv', index=False)
print(wordsDF.to_string)

<bound method DataFrame.to_string of              word   ratio  cantidadLetras  gotAccent  doubleVocal  enCC  y
0             cat  0.3333               3          0            0     0  0
1          llibre  0.3333               6          0            0     0  0
2             any  0.3333               3          0            0     0  1
3        permetre  0.3750               8          0            0     0  0
4           plain  0.4000               5          0            0     0  1
...           ...     ...             ...        ...          ...   ... ..
1971       centre  0.3333               6          0            0     0  0
1972      between  0.4286               7          0            1     1  1
1973      solució  0.4286               7          1            0     0  0
1974         land  0.2500               4          0            0     0  1
1975  multiplicar  0.3636              11          0            0     0  0

[1976 rows x 7 columns]>


### Separación del conjunto de datos en entrenamiento y test
Vamos a escoger dos tercios para el conjunto de entrenamiento y el resto para el test. 

In [9]:
#First we must separate dataframe into X and y format
X=wordsDF.iloc[:,1:6]
X_aux=wordsDF.iloc[:,1:3]
y=wordsDF.iloc[:,6]
#For better perfomance, we scale the data using an standard scaler
scaler = sklearn.preprocessing.StandardScaler()
scaler.fit(X_aux)
standardData_aux = scaler.transform(X_aux)
print(standardData_aux)
#Ratio Standarized
print(standardData_aux[:,0])
#NumLletres Standarized
print(standardData_aux[:,1])
standardData=X
standardData['ratio']=standardData_aux[:,0]
standardData['cantidadLetras']=standardData_aux[:,1]
print(standardData)
#Then we separate the data frame in training and test (will be used in chosen model)
X_train, X_test, y_train, y_test = train_test_split(standardData, y, test_size=0.33, random_state=33)

[[-0.40249248 -1.20923579]
 [-0.40249248  0.4099984 ]
 [-0.40249248 -1.20923579]
 ...
 [ 0.37096326  0.94974313]
 [-1.07855609 -0.66949106]
 [-0.15657738  3.10872205]]
[-0.40249248 -0.40249248 -0.40249248 ...  0.37096326 -1.07855609
 -0.15657738]
[-1.20923579  0.4099984  -1.20923579 ...  0.94974313 -0.66949106
  3.10872205]
         ratio  cantidadLetras  gotAccent  doubleVocal  enCC
0    -0.402492       -1.209236          0            0     0
1    -0.402492        0.409998          0            0     0
2    -0.402492       -1.209236          0            0     0
3    -0.064055        1.489488          0            0     0
4     0.138845       -0.129746          0            0     0
...        ...             ...        ...          ...   ...
1971 -0.402492        0.409998          0            0     0
1972  0.370963        0.949743          0            1     1
1973  0.370963        0.949743          1            0     0
1974 -1.078556       -0.669491          0            0     0
197

## Modelos
En este problema creemos que el mejor modelo para clasificar las palabras va a ser una SVC con un kernel gaussiano (RBG) ya que tenemos un número de características bastante bajo en comparación al número de muestras. 

Los híperparámetros que deberemos de configurar en el modelo del SVM Lineal son:
- _C_
- _max\_iter_
- _gamma_

El resto de híperparámetros no son tan necesarios ya que no se ajustan al problema.

In [10]:
#Modelo
SVM_rbf = SVC(kernel='rbf')

## Nested Cross-Validation
Para el Nested Cross-Validation haremos lo siguiente. 

In [11]:
hyper_grid = {"C":[0.1,1,10],"max_iter":[100,1000,10000],"gamma":[.01,.1,1]}

inner_cv=StratifiedKFold(n_splits=3, shuffle=False)
outer_cv=StratifiedKFold(n_splits=5, shuffle=False)

search=GridSearchCV(estimator=SVM_rbf,cv=inner_cv,param_grid=hyper_grid,n_jobs=-1)
search.fit(X_train,y_train)
test_score=cross_val_score(search,X_train,y_train,cv=outer_cv,n_jobs=-1)
print(f"Mean score of Nested Cross Validation:"f"{test_score.mean():.3f}+-{test_score.std():.3f}")
print(f"The best estimator is:"f"{search.best_estimator_}")

Mean score of Nested Cross Validation:0.644+-0.014
The best estimator is:SVC(C=1, gamma=1, max_iter=1000)


## Prueba del Modelo con los parámetros encontrados

Una vez hemos encontrado los mejores híperparámetros para el SVM con Kernel Gaussiano, procedemos a entrenarlo y a realizar la predicción.

In [12]:
SVM_rbf.fit(X_train,y_train)
y_predicted=SVM_rbf.predict(X_test)
errores=y_predicted-y_test
print(errores)

822    -1
142     0
1648    0
1622    0
1214    1
       ..
1838    0
1187    0
1649   -1
816     0
252     0
Name: y, Length: 653, dtype: int64
